In [1]:
import torch
import torch.nn as nn

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator

from einops import rearrange, repeat, reduce

from darts.datasets import AirPassengersDataset

In [38]:
import numpy as np
import matplotlib.pyplot as plt

def get_position_encoding(seq_len, d_model):
    position_encoding = np.zeros((seq_len, 1, d_model))
    for pos in range(seq_len):
        for i in range(d_model):
            if i % 2 == 0:
                position_encoding[pos, i] = np.sin(pos / (10000 ** (i / d_model)))
            else:
                position_encoding[pos, i] = np.cos(pos / (10000 ** ((i - 1) / d_model)))
    return position_encoding

# Parameters
seq_len = 1000  # Length of the sequence
d_model = 500  # Dimension of the model

# Generate position encoding
pos_encoding = get_position_encoding(seq_len, d_model)

# # Visualize the position encoding
# plt.figure(figsize=(10, 8))
# plt.pcolormesh(pos_encoding, cmap='viridis')
# plt.xlabel('Depth')
# plt.ylabel('Position')
# plt.colorbar()
# plt.title('Position Encoding Visualization')
# plt.show()


IndexError: index 1 is out of bounds for axis 1 with size 1

In [56]:
import math

position = torch.arange(10).unsqueeze(1)
div_term = torch.exp(torch.arange(0, 5, 2) * (-math.log(10000.0) / 5))
pe = torch.zeros(10, 5)
pe[:, 0::2] = torch.sin(position * div_term)
pe[:, 1::2] = torch.cos(position * div_term)

RuntimeError: The expanded size of the tensor (2) must match the existing size (3) at non-singleton dimension 1.  Target sizes: [10, 2].  Tensor sizes: [10, 3]

In [36]:
position = torch.arange(10).unsqueeze(1)
position

tensor([[0],
        [1],
        [2],
        [3],
        [4],
        [5],
        [6],
        [7],
        [8],
        [9]])

In [55]:
div_term = torch.arange(0, 5, 2) * (-math.log(10000.0) / 5)

div_term

tensor([-0.0000, -3.6841, -7.3683])

In [38]:
pe = torch.zeros(10,5)
pe

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [39]:
pe[:, 0::2] = torch.sin(position * div_term)
pe

tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 8.4147e-01,  0.0000e+00,  2.5116e-02,  0.0000e+00,  6.3096e-04],
        [ 9.0930e-01,  0.0000e+00,  5.0217e-02,  0.0000e+00,  1.2619e-03],
        [ 1.4112e-01,  0.0000e+00,  7.5285e-02,  0.0000e+00,  1.8929e-03],
        [-7.5680e-01,  0.0000e+00,  1.0031e-01,  0.0000e+00,  2.5238e-03],
        [-9.5892e-01,  0.0000e+00,  1.2526e-01,  0.0000e+00,  3.1548e-03],
        [-2.7942e-01,  0.0000e+00,  1.5014e-01,  0.0000e+00,  3.7857e-03],
        [ 6.5699e-01,  0.0000e+00,  1.7493e-01,  0.0000e+00,  4.4167e-03],
        [ 9.8936e-01,  0.0000e+00,  1.9960e-01,  0.0000e+00,  5.0476e-03],
        [ 4.1212e-01,  0.0000e+00,  2.2415e-01,  0.0000e+00,  5.6786e-03]])

In [51]:
pe[:, 1::2] = torch.cos(position * div_term)
pe

RuntimeError: The expanded size of the tensor (2) must match the existing size (3) at non-singleton dimension 1.  Target sizes: [10, 2].  Tensor sizes: [10, 3]

In [71]:
abc = torch.rand(4,3)
abc

tensor([[0.2049, 0.9688, 0.3221],
        [0.5689, 0.0773, 0.0548],
        [0.9827, 0.5015, 0.5390],
        [0.7882, 0.2086, 0.3103]])

In [72]:

xyz = torch.fft.fft(abc, dim=0)
xyz

tensor([[ 2.5447+0.0000j,  1.7561+0.0000j,  1.2262+0.0000j],
        [-0.7778+0.2193j,  0.4672+0.1313j, -0.2169+0.2555j],
        [-0.1696+0.0000j,  1.1844+0.0000j,  0.4961+0.0000j],
        [-0.7778-0.2193j,  0.4672-0.1313j, -0.2169-0.2555j]])

In [74]:
x = torch.fft.ifft(xyz)
x.real

tensor([[ 1.8424,  0.3512,  0.3512],
        [-0.1758, -0.2652, -0.3368],
        [ 0.5037, -0.3366, -0.3366],
        [-0.1758, -0.3368, -0.2652]])

In [12]:
X = torch.rand(25, 10)
W = torch.rand(10, 25)
A = torch.rand(10,2)

In [13]:
m = torch.matmul(W,X)
m.shape

torch.Size([10, 10])

In [14]:
torch.matmul(m,A).shape

torch.Size([10, 2])

In [18]:
abc = torch.tensor([1,2,3,4,5,6,7])
abc

tensor([1, 2, 3, 4, 5, 6, 7])

In [25]:
torch.flip(abc, [0])

tensor([7, 6, 5, 4, 3, 2, 1])

In [26]:
abc = torch.rand(5,2,1)
abc

tensor([[[0.3241],
         [0.2891]],

        [[0.4195],
         [0.8966]],

        [[0.4549],
         [0.6025]],

        [[0.4123],
         [0.8547]],

        [[0.3778],
         [0.2365]]])

In [27]:
torch.flip(abc, [0])

tensor([[[0.3778],
         [0.2365]],

        [[0.4123],
         [0.8547]],

        [[0.4549],
         [0.6025]],

        [[0.4195],
         [0.8966]],

        [[0.3241],
         [0.2891]]])

In [28]:
torch.flip(abc, [1])

tensor([[[0.2891],
         [0.3241]],

        [[0.8966],
         [0.4195]],

        [[0.6025],
         [0.4549]],

        [[0.8547],
         [0.4123]],

        [[0.2365],
         [0.3778]]])

In [36]:
input = torch.LongTensor([[1,2,3],[5,6, 7]])
input

tensor([[1, 2, 3],
        [5, 6, 7]])

In [35]:
embedding = nn.Embedding(10,3)
embedding(input)

RuntimeError: Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.FloatTensor instead (while checking arguments for embedding)

In [36]:
batch=501
seq=10
abc = torch.rand(batch,10)
abc.shape

torch.Size([501, 10])

In [37]:
abc = abc.unsqueeze(1)
abc.shape

torch.Size([501, 1, 10])

In [40]:

abc.view(torch.round(abc.shape[0]//seq), seq, 10).shape

TypeError: round(): argument 'input' (position 1) must be Tensor, not int